# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
from sql_queries import *

In [2]:
%run create_tables.py

In [3]:
# conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
# cur = conn.cursor()
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get cursor to the Database")
    print(e)
conn.set_session(autocommit=True)

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('data/song_data/')
filepath = song_files[0]
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR051KA1187B98B2FF,NaN,,NaN,Wilks,261.51138,1,SOLYIBD12A8C135045,Music is what we love,0


## #1: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

I've got column constraints that require an artist to exist before any of their songs can exist (as it really should be for a proper relational database,) so the artist record must be created before the song record.

In [6]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values.tolist()[0]
print(artist_data)

['AR051KA1187B98B2FF', 'Wilks', '', nan, nan]


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [7]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()[0]
print(song_data)

['SOLYIBD12A8C135045', 'Music is what we love', 'AR051KA1187B98B2FF', 0, 261.51138]


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2.5: `artists` & `songs` Table Reprise
Populate them

In [10]:
for filepath in song_files:
    # read in the data
    df = pd.read_json(filepath, lines=True)

    # create the artist record data
    artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values.tolist()[0]
    
    # create the song record data
    song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()[0]
    
    # insert the artist and song records
    cur.execute(artist_table_insert, artist_data)
    cur.execute(song_table_insert, song_data)
    conn.commit()

In [11]:
'''
songs
      song_id
    , title
    , artist_id
    , year
    , duration

artists
      artist_id
    , name
    , location
    , latitude
    , longitude

'''
sna  = ("""
SELECT
      s.title
    , s.song_id
    , s.artist_id
    , a.name
FROM songs AS s
    JOIN artists AS a ON a.artist_id = s.artist_id
LIMIT 10
""")

'''
%sql select s.song_id, a.artist_id FROM songs as s join artists as a on s.artist_id = a.artist_id where s.title = 'Ten Tonne' and a.name = 'Chase & Status' and s.duration = 337.68444
'''

try:
    cur.execute(sna)
    rows = cur.fetchall()
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

('Music is what we love', 'SOLYIBD12A8C135045', 'AR051KA1187B98B2FF', 'Wilks')
('Native Soul', 'SONSKXP12A8C13A2C9', 'AR0IAWL1187B9A96D0', 'Danilo Perez')
('A Whiter Shade Of Pale (Live @ Fillmore West)', 'SODREIN12A58A7F2E5', 'ARLTWXK1187FB5A3F8', 'King Curtis')
('Der Kleine Dompfaff', 'SOUPIRU12A6D4FA1E1', 'ARJIE2Y1187B994AB7', 'Line Renaud')
('Prognosis', 'SODAUVL12A8C13D184', 'ARWB3G61187FB49404', 'Steve Morse')
('City Slickers', 'SOINLJW12A8C13314C', 'AR8IEZO1187B99055E', 'Marc Shaiman')
('Streets On Fire (Explicit Album Version)', 'SOWQTQZ12A58A7B63E', 'ARPFHN61187FB575F6', 'Lupe Fiasco')
('The Ballad Of Sleeping Beauty', 'SONWXQJ12A8C134D94', 'ARNF6401187FB57032', 'Sophie B. Hawkins')
('Living Hell', 'SOFCHDR12AB01866EF', 'AREVWGE1187B9B890A', 'Bitter End')
('Intro', 'SOGDBUF12A8C140FAA', 'AR558FS1187FB45658', '40 Grit')


Run `test.ipynb` to see if you've successfully added records to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = get_files('data/log_data')
filepath = log_files[0]
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
1,Beastie Boys,Logged In,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Lighten Up,200,1541722186796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
2,Shakira,Logged In,Harper,M,3,Barrett,145.84118,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Pienso En Ti,200,1541722347796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
3,Selena,Logged In,Harper,M,4,Barrett,172.66893,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Amor Prohibido,200,1541722492796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
4,Kid Cudi Vs Crookers,Logged In,Harper,M,5,Barrett,162.97751,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Day 'N' Nite,200,1541722664796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [13]:
df = df.query(" page == 'NextSong' ")
t = pd.to_datetime(df['ts'], unit='ms')
df['ts'] = pd.to_datetime(df['ts'])
time_data = (t, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday)
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')
time_df = pd.concat(time_data, axis=1, keys=column_labels)
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-09 00:06:17.796,0,9,45,11,2018,4
1,2018-11-09 00:09:46.796,0,9,45,11,2018,4
2,2018-11-09 00:12:27.796,0,9,45,11,2018,4
3,2018-11-09 00:14:52.796,0,9,45,11,2018,4
4,2018-11-09 00:17:44.796,0,9,45,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [14]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [15]:
dfuser = df[df['userId'] != '']
user_df = dfuser[['userId', 'firstName', 'lastName', 'gender', 'level']]
# user_df

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [16]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [17]:
song = 'Endlessly'
artist = 'Muse'
duration = '228.93669'

for index, row in df.iterrows():
    
    # get songid and artistid from song and artist tables
    # print("Song:", row.song, "  Artist:", row.artist, "  Duration:", row.length)
    cur.execute(song_select, (row.song, row.artist, row.length))
    # print("Song:", song, "  Artist:", artist, "  Duration:", duration)
    # cur.execute(song_select, (song, artist, duration))
    results = cur.fetchall()
    # print(results) # all returned "None"
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    start_time = pd.to_datetime(row['ts'], unit='ms')
    songplay_data = (start_time, int(row.userId), row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


Run `test.ipynb` to see if you've successfully added records to this table.

This is not pulling the artist and song data correctly.

# Close Connection to Sparkify Database

In [18]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.